In [1]:
import pandas as pd
import json

#  Cargar el CSV con pandas
ruta_csv = "D:/Soy Henry/Proyecto Final/metadata.csv"  # Ajusta la ruta real
df = pd.read_csv(ruta_csv)

#  Manejo de valores nulos
df.fillna({
    'description': 'Desconocido',  # Rellenar descripciones vacías
    'address': 'Sin dirección'  # Rellenar direcciones vacías
}, inplace=True)

#  Convertir columnas numéricas al tipo correcto
df["avg_rating"] = pd.to_numeric(df["avg_rating"], errors="coerce")
df["num_of_reviews"] = pd.to_numeric(df["num_of_reviews"], errors="coerce", downcast="integer")

#  Transformar 'category' de JSON-string a CSV-compatible
def clean_json_column(json_str):
    try:
        return ",".join(json.loads(json_str.replace("'", '"'))) if pd.notna(json_str) else ""
    except json.JSONDecodeError:
        return ""

df["category"] = df["category"].apply(clean_json_column)

# Parsear JSON en 'hours' y 'MISC'
def parse_json_safe(json_str):
    try:
        return json.dumps(json.loads(json_str.replace("'", '"'))) if pd.notna(json_str) else "{}"
    except json.JSONDecodeError:
        return "{}"

df["MISC"] = df["MISC"].apply(parse_json_safe)

#  Contar valores nulos
print(df.isnull().sum())
#  Estadísticas básicas de ratings
print(df["avg_rating"].describe())

#codigo para eliminar estas columnas:gmap_id,description,latitude,longitude,price,state,relative_results ,hours,url

columnas_a_eliminar = ["description", "latitude", "longitude", "price", "state", "relative_results","hours","url"]
df.drop(columns=columnas_a_eliminar, inplace=True)


name                     38
address                   0
gmap_id                   0
description               0
latitude                  0
longitude                 0
category                  0
avg_rating                0
num_of_reviews            0
price               1741198
hours                491188
MISC                      0
state                465999
relative_results     182620
url                       0
dtype: int64
count    1.925007e+06
mean     4.302853e+00
std      7.051855e-01
min      1.000000e+00
25%      4.000000e+00
50%      4.500000e+00
75%      4.800000e+00
max      5.000000e+00
Name: avg_rating, dtype: float64


Transformar MISC en una lista legible
En lugar de convertir MISC en varias columnas, podemos simplemente extraer los valores del diccionario y guardarlos como una lista.

In [2]:
import json

# Función para convertir el JSON en una lista de palabras clave
def transformar_misc(misc_str):
    try:
        # Convertir de string a diccionario
        misc_dict = json.loads(misc_str.replace("'", '"')) if pd.notna(misc_str) else {}
        
        # Extraer todos los valores en una lista
        keywords = []
        for values in misc_dict.values():
            if isinstance(values, list):  # Si el valor es una lista, agregar todos los elementos
                keywords.extend(values)
            else:  # Si es un solo valor, agregarlo directamente
                keywords.append(values)
        
        return ", ".join(keywords)  # Unir en un solo string separado por comas
    except json.JSONDecodeError:
        return ""

# Aplicar la transformación a la columna MISC
df["MISC"] = df["MISC"].apply(transformar_misc)


In [3]:
#  Guardar en **CSV optimizado para Power BI**
ruta_salida_csv = "D:\Soy Henry\Proyecto Final\metadata_clean.csv"
df.to_csv(ruta_salida_csv, index=False, encoding="utf-8")
print("✅ ETL finalizado con pandas. CSV listo para Power BI.")

<>:2: SyntaxWarning: invalid escape sequence '\S'
<>:2: SyntaxWarning: invalid escape sequence '\S'
C:\Users\calde\AppData\Local\Temp\ipykernel_8464\137123353.py:2: SyntaxWarning: invalid escape sequence '\S'
  ruta_salida_csv = "D:\Soy Henry\Proyecto Final\metadata_clean.csv"


✅ ETL finalizado con pandas. CSV listo para Power BI.


🔹 Resumen de Cambios
✅ Manejo de valores nulos:

Se rellenaron valores nulos en description con "Desconocido".
Se rellenaron valores nulos en address con "Sin dirección".
✅ Conversión de tipos de datos:

latitude, longitude, avg_rating y num_of_reviews fueron convertidos a float o integer según corresponda.
num_of_reviews se optimizó con downcast="integer" para reducir uso de memoria.
✅ Transformaciones y limpieza de datos:

Se convirtió la columna category de JSON-string a formato compatible con CSV.
Se limpiaron caracteres no deseados en hours (por ejemplo, "â€“" → "-").
Se manejó el JSON en MISC, asegurando un formato válido.
✅ Optimización para Power BI:

Se guardó el archivo transformado como metadata_clean.csv en formato UTF-8.
✅ Análisis de calidad de datos:

Se realizó un conteo de valores nulos.
Se filtraron negocios por categoría específica (ejemplo: farmacias).
Se generaron estadísticas básicas de ratings (avg_rating).
Principales Mejoras:
🔹 Se asegura consistencia y compatibilidad de datos para análisis en Power BI.
🔹 Reducción de errores en JSON y caracteres extraños.
🔹 Se mejoró la eficiencia con tipos de datos correctos y optimización de memoria.
🔹 Código más robusto y reutilizable para futuras transformaciones.

✅ ETL Google Maps finalizado con éxito.